## Launch an EC2 webserver on AWS

In this workshop we will explore the basics of AWS with VPC, EC2, and Amazon S3. Python is used extensively so you will need experience in or be comfortable reading python code. 

### Initialize notebook

We will be using the [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) library for creation of all resources.

In [ ]:
import boto3
import sys
import os
import json
import base64
import pprint
import uuid
from dateutil import parser

#from lib import workshop
from botocore.exceptions import ClientError

ec2_client = boto3.client('ec2')
ec2 = boto3.resource('ec2')
cloudwatch = boto3.client('cloudwatch')

session = boto3.session.Session()
region = session.region_name

sec_group_name = 'web-sg'

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = ''.join(['intro-', str(uuid.uuid4())])
session.resource('s3').create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
print(bucket)

### [Create VPC](https://aws.amazon.com/vpc/)

Amazon Virtual Private Cloud (Amazon VPC) lets you provision a logically isolated section of the AWS Cloud where you can launch AWS resources in a virtual network that you define. You have complete control over your virtual networking environment, including selection of your own IP address range, creation of subnets, and configuration of route tables and network gateways. You can use both IPv4 and IPv6 in your VPC for secure and easy access to resources and applications.

In [ ]:
vpc = ec2.create_vpc(CidrBlock='10.0.0.0/16')
vpc.modify_attribute(EnableDnsSupport={'Value':True})
vpc.modify_attribute(EnableDnsHostnames={'Value':True})
tag = vpc.create_tags(
Tags=[
    {
        'Key': 'Name',
        'Value': 'research-workshop'
    },
])

subnet = vpc.create_subnet(CidrBlock='10.0.0.0/24', AvailabilityZone=region + 'a')
subnet.meta.client.modify_subnet_attribute(
    SubnetId=subnet.id, 
    MapPublicIpOnLaunch={"Value": True}
)

subnet2 = vpc.create_subnet(CidrBlock='10.0.1.0/24', AvailabilityZone=region + 'b')
subnet2.meta.client.modify_subnet_attribute(SubnetId=subnet2.id, MapPublicIpOnLaunch={"Value": True})

igw = ec2.create_internet_gateway()
igw.attach_to_vpc(VpcId=vpc.id)

public_route_table = list(vpc.route_tables.all())[0]
# add a default route, for Public Subnet, pointing to Internet Gateway 
ec2_client.create_route(RouteTableId=public_route_table.id,DestinationCidrBlock='0.0.0.0/0',GatewayId=igw.id)
public_route_table.associate_with_subnet(SubnetId=subnet.id)
public_route_table.associate_with_subnet(SubnetId=subnet2.id)

vpc_id = vpc.id
subnet_id = subnet.id
subnet2_id = subnet2.id
print(vpc_id)
print(subnet_id)
print(subnet2_id)

### Create index.html page for the web application

We will write out a simple html page to demo setting up the Apache web server using an Application Load Balancer and Auto Scaling to provide elasticity to your web application.

In [ ]:
%%writefile index.html

<h1>Hello from the intro to AWS workshop!!!</h1>

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Next, we will upload the index.html file created above to S3 to be used later in the workshop.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
session.resource('s3').Bucket(bucket).Object(os.path.join('web', 'index.html')).upload_file('index.html')

### [Create Security Groups](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_SecurityGroups.html)


A security group acts as a virtual firewall for your instance to control inbound and outbound traffic. When you launch an instance in a VPC, you can assign up to five security groups to the instance. Security groups act at the instance level, not the subnet level. Therefore, each instance in a subnet in your VPC could be assigned to a different set of security groups. If you don't specify a particular group at launch time, the instance is automatically assigned to the default security group for the VPC.

[ec2_client.create_security_group](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.create_security_group) boto3 documentation

In [ ]:
sg = ec2_client.create_security_group(
    Description='security group for EC2 instance',
    GroupName=sec_group_name,
    VpcId=vpc_id
)
sec_group_id=sg["GroupId"]
print('EC2 Security group id - ' + sec_group_id)

### Configure available ports

In order for the EC2 Instance to communicate with the outside world, we will open port 80 and 443. As you can see in the call below we can define the `ToPort` and `FromPort` and a `CidrIp` range we want to allow.

[ec2_client.authorize_security_group_ingress](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.authorize_security_group_ingress) boto3 documentation

In [ ]:
data = ec2_client.authorize_security_group_ingress(
    GroupId=sec_group_id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
         'FromPort': 80,
         'ToPort': 80,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'HTTP access'
            },
          ]
        },
        {'IpProtocol': 'tcp',
         'FromPort': 443,
         'ToPort': 443,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'HTTPS access'
            },
          ]
        }
    ]
)

### Get Latest Amazon Linux AMI

An Amazon Machine Image (AMI) is a template that contains a software configuration (for example, an operating system, an application server, and applications). From an AMI, you launch an instance, which is a copy of the AMI running as a virtual server in the cloud.

Here we find the latest Amazon Linux AMI that is compatitible with the instance we will run later. 

In [ ]:
filters = [ {
    'Name': 'name',
    'Values': ['amzn-ami-hvm-*']
},{
    'Name': 'description',
    'Values': ['Amazon Linux AMI*']
},{
    'Name': 'architecture',
    'Values': ['x86_64']
},{
    'Name': 'owner-alias',
    'Values': ['amazon']
},{
    'Name': 'owner-id',
    'Values': ['137112412989']
},{
    'Name': 'state',
    'Values': ['available']
},{
    'Name': 'root-device-type',
    'Values': ['ebs']
},{
    'Name': 'virtualization-type',
    'Values': ['hvm']
},{
    'Name': 'hypervisor',
    'Values': ['xen']
},{
    'Name': 'image-type',
    'Values': ['machine']
} ]

response = ec2_client.describe_images(Owners=['amazon'], Filters=filters)

# Search for the newest image
latest = None
for image in response['Images']:
    if not latest:
        latest = image
        continue
    if parser.parse(image['CreationDate']) > parser.parse(latest['CreationDate']):
        latest = image

ami=latest['ImageId'] 
print(ami)

### Create UserData to install Apache web server and download index

Replace the `{{bucket}}` value with the S3 bucket you created above

In [ ]:
%%writefile userdata.sh

#!/bin/bash
yum update -y
yum -y install httpd
service httpd start

usermod -a -G apache ec2-user
chown -R ec2-user:apache /var/www
chmod 2775 /var/www
find /var/www -type d -exec chmod 2775 {} \;
find /var/www -type f -exec chmod 0664 {} \;

aws s3 cp s3://intro-8c7b22c0-31cd-4b8a-a485-6da532f2ca00/web/index.html /var/www/html/index.html

### Load userdata.sh

We will read the UserData into a local variable and base64 encode the contents of the file to be used on the EC2 instance launch configuraton.

In [ ]:
fh=open("userdata.sh")
userdata=fh.read()
fh.close()

userdataencode = base64.b64encode(userdata.encode()).decode("ascii")

### [Create an EC2 Instance](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/LaunchingAndUsingInstances.html)

An instance is a virtual server in the cloud. Its configuration at launch is a copy of the AMI that you specified when you launched the instance.

You can launch different types of instances from a single AMI. An instance type essentially determines the hardware of the host computer used for your instance. Each instance type offers different compute and memory capabilities. Select an instance type based on the amount of memory and computing power that you need for the application or software that you plan to run on the instance. For more information about the hardware specifications for each Amazon EC2 instance type, see [Amazon EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/).

After you launch an instance, it looks like a traditional host, and you can interact with it as you would any computer. You have complete control of your instances; you can use sudo to run commands that require root privileges.

Your AWS account has a limit on the number of instances that you can have running. For more information about this limit, and how to request an increase, see [How many instances can I run in Amazon EC2](https://aws.amazon.com/ec2/faqs/#How_many_instances_can_I_run_in_Amazon_EC2)
in the Amazon EC2 General FAQ. 

[ec2.create_instances](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.ServiceResource.create_instances) boto3 documentation


In [ ]:
ec2_instance = ec2.create_instances(
    ImageId=ami,
    MinCount=1,
    MaxCount=1,
    InstanceType='m5.large', 
    Monitoring={'Enabled': True},
    UserData=userdataencode,
    NetworkInterfaces=[
        {
            'AssociatePublicIpAddress': True,
            'DeviceIndex': 0,
            'SubnetId': subnet_id,
            'Groups' : [
                sec_group_id,
            ]
        }
    ],
    TagSpecifications=[
        {
            'ResourceType':'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'research-demo'
                },
            ]
        },
    ],
)
print(ec2_instance[0].state)

## Validate web server

Wait for the EC2 launch to complete. This may take a couple of minutes.

In [ ]:
ec2_instance[0].wait_until_running()

ec2_instance[0].reload()
print("Instance id: {0}".format(ec2_instance[0].id))
print("State: {0}".format(ec2_instance[0].state['Name']))

In [ ]:
print("EC2 Instance {0}: https://{1}.console.aws.amazon.com/ec2/v2/home?region={1}#Instances:sort=instanceId".format(ec2_instance[0].id, region))
print("Web App: http://{0}".format(ec2_instance[0].public_dns_name))


## Finished!!!!!!

From the links above you can now click on the Web App link to launch a new tab in the browser to show the index.html page we uploaded from S3. After that, you can click the EC2 Instance link to look at your EC2 Instance. 

If you were to create this in a production environment you could leverage [CloudFormation](https://aws.amazon.com/cloudformation/) templates that will allow you to leverage YAML or JSON templates to launch the resources. If you would like to experiment more you can launch example [CloudFormation application templates](https://docs.aws.amazon.com/AWSCloudFormation/latest/UserGuide/deploying.applications.html) to see how you could build your [Infrastructure as Code](https://en.wikipedia.org/wiki/Infrastructure_as_code). 

## Clean Up

In order to remove everything created in this workshop you can run the cells below and finally remove the VPC created for this workshop.

### Remove EC2 Instance

In [ ]:
ec2_instance[0].terminate()
ec2_instance[0].wait_until_terminated()
ec2_instance[0].reload()
print("Complete Instance Stopped")

### Remove Security Group

In [ ]:
response = ec2_client.delete_security_group(GroupId=sec_group_id)

### Remove Virtual Private Cloud

In [ ]:
"""Cleanup VPC"""
print('Removing VPC ({}) from AWS'.format(vpc_id))
ec2 = boto3.resource('ec2')
ec2_client = ec2.meta.client
vpc = ec2.Vpc(vpc_id)

# detach default dhcp_options if associated with the vpc
dhcp_options_default = ec2.DhcpOptions('default')
if dhcp_options_default:
    dhcp_options_default.associate_with_vpc(
        VpcId=vpc.id
    )
# detach and delete all gateways associated with the vpc
for gw in vpc.internet_gateways.all():
    vpc.detach_internet_gateway(InternetGatewayId=gw.id)
    gw.delete()
# delete all route table associations
for rt in vpc.route_tables.all():
    if not rt.associations:
        rt.delete()
    else:
        for rta in rt.associations:
            if not rta.main:
                rta.delete()
# delete any instances
for subnet in vpc.subnets.all():
    for instance in subnet.instances.all():
        instance.terminate()
# delete our endpoints
for ep in ec2_client.describe_vpc_endpoints(
        Filters=[{
            'Name': 'vpc-id',
            'Values': [vpc_id]
        }])['VpcEndpoints']:
    ec2_client.delete_vpc_endpoints(VpcEndpointIds=[ep['VpcEndpointId']])
# delete our security groups
for sg in vpc.security_groups.all():
    if sg.group_name != 'default':
        sg.delete()
# delete any vpc peering connections
for vpcpeer in ec2_client.describe_vpc_peering_connections(
        Filters=[{
            'Name': 'requester-vpc-info.vpc-id',
            'Values': [vpc_id]
        }])['VpcPeeringConnections']:
    ec2.VpcPeeringConnection(vpcpeer['VpcPeeringConnectionId']).delete()
# delete non-default network acls
for netacl in vpc.network_acls.all():
    if not netacl.is_default:
        netacl.delete()
# delete network interfaces
for subnet in vpc.subnets.all():
    for interface in subnet.network_interfaces.all():
        interface.delete()
    subnet.delete()
# finally, delete the vpc
ec2_client.delete_vpc(VpcId=vpc_id)
print('VPC ({}) removed from AWS'.format(vpc_id))

### Remove S3 Bucket

In [ ]:
"""Remove all objects from S3 bucket and delete"""
client = boto3.client('s3')

response = client.list_objects_v2(
    Bucket=bucket,
)

while response['KeyCount'] > 0:
    print('Deleting %d objects from bucket %s' % (len(response['Contents']),bucket))
    response = client.delete_objects(
        Bucket=bucket,
        Delete={
            'Objects':[{'Key':obj['Key']} for obj in response['Contents']]
        }
    )
    response = client.list_objects_v2(
        Bucket=bucket,
    )

print('Now deleting bucket %s' % bucket)
response = client.delete_bucket(
    Bucket=bucket
)